[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/mutual-fund-holdings-nport-python.ipynb)

# SEC Mutual Fund Holdings (N-PORT) with Python -- Free, No API Key

Use **edgartools** to analyze mutual fund and ETF portfolio holdings from SEC N-PORT filings -- completely free, no API key or paid subscription required. Every mutual fund and ETF must file an N-PORT monthly report disclosing every security it holds, including stocks, bonds, and derivatives.

**What you'll learn:**
- Parse an N-PORT filing to see every holding in a fund
- Get fund metadata: net assets, total assets, number of positions
- Find the top holdings by portfolio weight
- Analyze asset allocation and country exposure
- Explore bond fund holdings with maturity and coupon data
- Compare portfolio composition across funds

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## See Every Holding in SPY

The SPDR S&P 500 ETF (SPY) files monthly N-PORT reports listing every stock it holds. Parse one to see the full portfolio:

In [ ]:
fund = Company("SPY").get_filings(form="NPORT-P")[0].obj()

print(f"Fund:        {fund.name}")
print(f"Period:      {fund.reporting_period}")
print(f"Holdings:    {len(fund.investments):,}")
print(f"Net Assets:  ${fund.fund_info.net_assets:,.0f}")

# Get all holdings as a DataFrame
df = fund.investment_data()
df["value_usd"] = pd.to_numeric(df["value_usd"])
df["pct_value"] = pd.to_numeric(df["pct_value"])

# Top 10 holdings by value
top = df.nlargest(10, "value_usd")[["name", "ticker", "value_usd", "pct_value"]].copy()
top["Value ($B)"] = (top["value_usd"] / 1e9).round(1)
top["Weight %"] = top["pct_value"].round(2)
top[["name", "ticker", "Value ($B)", "Weight %"]]

## Individual Holding Details

Each holding has detailed metadata including CUSIP, asset category, country, and more:

In [ ]:
holding = fund.investments[0]

print(f"Name:          {holding.name}")
print(f"Title:         {holding.title}")
print(f"Ticker:        {holding.ticker}")
print(f"CUSIP:         {holding.cusip}")
print(f"Shares:        {holding.balance:,.0f}")
print(f"Value (USD):   ${holding.value_usd:,.0f}")
print(f"Weight:        {holding.pct_value}%")
print(f"Asset Type:    {holding.asset_category}")
print(f"Issuer Type:   {holding.issuer_category}")
print(f"Country:       {holding.investment_country}")
print(f"Currency:      {holding.currency_code}")

## Country Exposure

N-PORT filings include the country of each holding, so you can analyze geographic exposure:

In [ ]:
countries = df.groupby("investment_country")["value_usd"].agg(["sum", "count"])
countries.columns = ["Total Value", "Holdings"]
countries = countries.sort_values("Total Value", ascending=False)
countries["Value ($B)"] = (countries["Total Value"] / 1e9).round(1)
countries["% of Fund"] = (countries["Total Value"] / countries["Total Value"].sum() * 100).round(1)
countries[["Holdings", "Value ($B)", "% of Fund"]]

## Bond Fund Holdings

N-PORT data is especially valuable for bond funds where holdings aren't publicly visible. Bond holdings include maturity dates, coupon rates, and asset categories:

In [ ]:
bond_fund = Company("BND").get_filings(form="NPORT-P")[0].obj()

print(f"Fund:       {bond_fund.name}")
print(f"Holdings:   {len(bond_fund.investments):,}")
print(f"Net Assets: ${bond_fund.fund_info.net_assets:,.0f}\n")

bdf = bond_fund.investment_data()
bdf["value_usd"] = pd.to_numeric(bdf["value_usd"])

# Asset type breakdown
cats = bdf.groupby("asset_category")["value_usd"].agg(["sum", "count"])
cats.columns = ["Total Value", "Holdings"]
cats = cats.sort_values("Total Value", ascending=False)
cats["Value ($B)"] = (cats["Total Value"] / 1e9).round(1)
cats[["Holdings", "Value ($B)"]]

## Top Bond Holdings with Maturity and Coupon

See the largest individual bond positions with their maturity dates and interest rates:

In [ ]:
bdf["pct_value"] = pd.to_numeric(bdf["pct_value"])
bdf["annualized_rate"] = pd.to_numeric(bdf["annualized_rate"])

top_bonds = bdf.nlargest(10, "value_usd")[["name", "value_usd", "pct_value", "maturity_date", "annualized_rate"]].copy()
top_bonds["Value ($B)"] = (top_bonds["value_usd"] / 1e9).round(1)
top_bonds["Weight %"] = top_bonds["pct_value"].round(2)
top_bonds["Coupon %"] = top_bonds["annualized_rate"].round(2)
top_bonds["Maturity"] = pd.to_datetime(top_bonds["maturity_date"]).dt.strftime("%Y-%m-%d")
top_bonds[["name", "Value ($B)", "Weight %", "Maturity", "Coupon %"]]

## Compare Fund Portfolios

Pull N-PORT data from multiple funds to compare portfolio characteristics side by side:

In [ ]:
tickers = ["SPY", "QQQ", "BND"]
rows = []

for ticker in tickers:
    fund = Company(ticker).get_filings(form="NPORT-P")[0].obj()
    df = fund.investment_data()
    df["value_usd"] = pd.to_numeric(df["value_usd"])
    top_cat = df.groupby("asset_category")["value_usd"].sum().idxmax()

    rows.append({
        "Ticker": ticker,
        "Fund": fund.name.split(" - ")[0][:35],
        "Holdings": f"{len(fund.investments):,}",
        "Net Assets ($B)": f"{float(fund.fund_info.net_assets) / 1e9:,.0f}",
        "Top Asset Type": top_cat,
    })

pd.DataFrame(rows).set_index("Ticker")

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing N-PORT fund holdings:

**With edgartools (free, no API key):**
```python
fund = Company("SPY").get_filings(form="NPORT-P")[0].obj()
fund.investment_data()              # All holdings as DataFrame
fund.fund_info.net_assets           # Fund net assets
holding.ticker                      # Resolved ticker for each position
```

**Typical approach (manual XML parsing):**
```python
from lxml import etree
# ... download N-PORT XML filing from EDGAR,
# ... parse dozens of XML namespaces and nested elements,
# ... map CUSIP to tickers manually, handle bond vs equity vs derivative structures
```

With edgartools, N-PORT data is parsed automatically -- every holding with ticker, value, weight, maturity, and coupon accessible as a pandas DataFrame.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Parse an N-PORT filing ──
fund = Company("SPY").get_filings(form="NPORT-P")[0].obj()

# ── Fund metadata ──
fund.name                              # Fund name
fund.reporting_period                  # Report date
fund.fund_info.net_assets              # Net asset value
fund.fund_info.total_assets            # Total assets
len(fund.investments)                  # Number of holdings

# ── Holdings as DataFrames ──
fund.investment_data()                 # All holdings
fund.securities_data()                 # Non-derivatives only
fund.derivatives_data()                # Derivatives only

# ── Individual holding ──
holding = fund.investments[0]
holding.name                           # Security name
holding.ticker                         # Ticker symbol
holding.cusip                          # CUSIP identifier
holding.value_usd                      # Market value (USD)
holding.pct_value                      # Portfolio weight %
holding.asset_category                 # EC (equity), DBT (debt), etc.
holding.investment_country             # Country code

# ── Bond-specific ──
holding.debt_security.maturity_date    # Maturity date
holding.debt_security.annualized_rate  # Coupon rate
holding.debt_security.coupon_kind      # Fixed or Floating
```

## What's Next

You've learned how to analyze mutual fund and ETF holdings from SEC N-PORT filings. Here are related tutorials:

- [Analyze ETF and Fund Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/etf-fund-holdings-python.ipynb)
- [Analyze 13F Institutional Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/13f-institutional-holdings-python.ipynb)
- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/financial-statements-sec-python.ipynb)
- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*